1. Instalar bibliotecas não-nativas

In [1]:
%pip install requests 
%pip install plyer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


2. Importar bibliotecas

In [2]:
from plyer import notification 
from datetime import datetime
import pandas as pd 
import requests 

### ETL

- Base de dados "Corretoras"

In [14]:
base = "CORRETORAS"
etapa = "EXTRACAO"

# Extract
def corretoras_database():
    try:
        url1 = "https://brasilapi.com.br/api/cvm/corretoras/v1"
        response = requests.get(url1, timeout=3)

        if response.status_code == 200:
            data_corretoras = response.json()
            corretoras_full = pd.DataFrame(data_corretoras)
            return corretoras_full
    
        else:
            today = datetime.today()
            notification.notify(

                    title="ATENÇÃO: Alerta Baixo",

                    message=f"Falha no carregamento da base {base} na etapa {etapa}\n{today}",

                    timeout=10 )   
    except requests.exceptions.Timeout:
        print("This request has timeout, try again later")

#Load
corretoras_database().to_excel("corretoras_database.xlsx", index = False)



- Base de dados "CNPJ"

In [16]:
base = "CNPJ"
etapa = "EXTRACAO"

#Extract

def cnpj_database():
    lista_cnpj = corretoras_database()['cnpj']
    cnpj_full = []

    for cnpj in lista_cnpj:
        try:
            url = 'https://brasilapi.com.br/api/cnpj/v1/{}'.format(cnpj)
            response = requests.get(url, timeout=3)
        except requests.exceptions.Timeout:
            continue
            
        if response.status_code == 200:
            data_cnpj = response.json()
            cnpj_full.append([data_cnpj['uf'], data_cnpj['cep'], data_cnpj['cnpj'],
                            data_cnpj['porte'],data_cnpj['numero'],data_cnpj['bairro'],
                            data_cnpj['ddd_fax'],data_cnpj['cnae_fiscal'],data_cnpj['complemento'],
                            data_cnpj['razao_social'],data_cnpj['nome_fantasia'],data_cnpj['ddd_telefone_1'],
                            data_cnpj['ddd_telefone_2'],data_cnpj['opcao_pelo_mei'],data_cnpj['codigo_municipio'],
                            data_cnpj['natureza_juridica'],data_cnpj['data_opcao_pelo_mei'],data_cnpj['data_exclusao_do_mei'],
                            data_cnpj['data_inicio_atividade'],data_cnpj['data_situacao_especial'],data_cnpj['opcao_pelo_simples'],
                            data_cnpj['data_opcao_pelo_simples'],data_cnpj['data_exclusao_do_simples'],data_cnpj['descricao_situacao_cadastral']])
        else:
            today = datetime.today()
            notification.notify(

                title="ATENÇÃO: Alerta Baixo",

                message=f"Falha no carregamento da base {base} na etapa {etapa}\n{today}",

                timeout=10 )

# Transform
            
    cnpj_full = pd.DataFrame(cnpj_full, columns=["UF","CEP","CNPJ","Porte","Numero","Bairro","Fax","CNAE","Complemento","Razao Social",
                                                "Nome Fantasia","Telefone 1","Telefone 2","Opcao pelo MEI","Codigo Municipio","Natureza Juridica",
                                                "Data Opcao pelo MEI","Data Exclusao MEI","Data Inicio da Atividade","Data Situacao Especial",
                                                "Opcao pelo Simples","Data Opcao pelo Simples","Data Exclusao do Simples","Descricao Situacao Cadastral"])
    return cnpj_full

# Load 

cnpj_database().to_excel("cnpj_database.xlsx", index = False)


- Base de dados "CEP"

In [17]:

base = "CEP"
etapa = "EXTRACAO"

#Extract

def cep_database():
    lista_cep = corretoras_database()['cep']
    cep_full = []

    for cep in lista_cep:
        try:
            url = 'https://brasilapi.com.br/api/cep/v2/{}'.format(cep)
            response = requests.get(url, timeout=3)
            
        except requests.exceptions.Timeout:
            continue    

        if response.status_code == 200:
            data_cep = response.json()
            cep_full.append([data_cep['cep'], data_cep['state'], data_cep['city'], data_cep['street']])
        else:
            today = datetime.today()
            notification.notify(

                title="ATENÇÃO: Alerta Baixo",

                message=f"Falha no carregamento da base {base} na etapa {etapa}\n{today}",

                timeout=10 )

        
# Transform
            
    cep_full = pd.DataFrame(cep_full, columns=["CEP","UF","Cidade","Logradouro"])
    return cep_full

# Load

cep_database().to_excel("cep_database.xlsx", index = False)

Versão das Bibliotecas

In [3]:
pip freeze

altair==4.2.2
anyio==3.6.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
attrs==22.2.0
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blinker==1.5
cachetools==5.3.0
certifi @ file:///C:/b/abs_85o_6fm0se/croot/certifi_1671487778835/work/certifi
cffi==1.15.1
charset-normalizer==3.0.1
click==8.1.3
colorama==0.4.6
comm==0.1.2
contourpy==1.0.7
cycler==0.11.0
debugpy==1.6.6
decorator==5.1.1
defusedxml==0.7.1
entrypoints==0.4
executing==1.2.0
fastjsonschema==2.16.3
fonttools==4.38.0
fqdn==1.5.1
gitdb==4.0.10
GitPython==3.1.31
idna==3.4
importlib-metadata==6.0.0
importlib-resources==5.12.0
ipykernel==6.21.2
ipython==8.10.0
ipython-genutils==0.2.0
ipywidgets==8.0.4
isoduration==20.11.0
jedi==0.18.2
Jinja2==3.1.2
joblib==1.2.0
jsonpointer==2.3
jsonschema==4.17.3
jupyter==1.0.0
jupyter-console==6.6.1
jupyter-events==0.6.3
jupyter_client==8.0.3
jupyter_core==5.2.0
jupyter_server==2.3.0
jupyter_server_terminals==0.4.4
jupyterlab-pygments==0.2.2
jupyterlab-widg